In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb
import holidays
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
#Lendo datasets
df_train = pd.read_csv('../data/raw/train.csv') 
#Convertendo coluna de datas
df_train['date'] = pd.to_datetime(df_train['date'])
df_train = df_train.set_index('date')
#Trabalhar com a cópia do dataframe
df_featured = df_train.copy()
df_featured.sort_index(inplace=True)

# Engenharia de Feature

## Features de Data

In [ ]:
df_featured['mes'] = df_featured.index.month
df_featured['dia_da_semana'] = df_featured.index.dayofweek # Segunda=0, Domingo=6
df_featured['dia_do_ano'] = df_featured.index.dayofyear
df_featured['semana_do_ano'] = df_featured.index.isocalendar().week.astype(int)
df_featured['ano'] = df_featured.index.year
# Feriados brasileiros
feriados_br = holidays.Brazil()
df_featured['e_feriado'] = df_featured.index.to_series().apply(lambda x: feriados_br.get(x)).notna().astype(int)

In [ ]:
df_featured

## Features de performance

In [ ]:
# Features de Lag vendas da semana anterior
df_featured['vendas_lag_7d'] = df_featured.groupby(['store', 'item'])['sales'].shift(7)

# Features de média de vendas da última semana
df_featured['media_vendas_7d'] = df_featured.groupby(['store', 'item'])['sales'].transform(
    lambda x: x.shift(1).rolling(window=7, min_periods=1).mean())

# Features de Lag de vendas da última quinzena
df_featured['vendas_lag_15d'] = df_featured.groupby(['store', 'item'])['sales'].shift(15)

# Features de média de vendas do última mês
df_featured['vendas_lag_30d'] = df_featured.groupby(['store', 'item'])['sales'].shift(30)

# Features de média de vendas da última semana
df_featured['media_vendas_30d'] = df_featured.groupby(['store', 'item'])['sales'].transform(
    lambda x: x.shift(1).rolling(window=30, min_periods=1).mean()
)

df_featured.dropna(inplace=True)

In [ ]:
df_featured.dtypes

In [ ]:
nome_arquivo = 'df_ML.csv'
df_featured.to_csv(nome_arquivo)

# Data Prep para Modelos de Machine Learning

In [ ]:
#Dividir em Treino e Validação
train = df_featured[df_featured.index.year < 2017] #Até 2016 para treino
valid = df_featured[df_featured.index.year == 2017] #2017 para validação

In [ ]:
#Definição de Features
x_train = train.drop('sales', axis=1)
y_train = train['sales']

x_valid = valid.drop('sales', axis=1)
y_valid = valid['sales']

# Verificar os formatos
print("Formato de X_treino:", x_train.shape)
print("Formato de X_validacao:", x_valid.shape)

# Alimentando Modelos de Machine Learning

In [ ]:
#Função de avaliação
def avaliar_modelo(modelo,x_train,y_train, x_test, y_test):
    """
    Calcula e exibe as métricas de avaliação (R², MAE, RMSE) para um modelo de regressão.
    """
    # Fazer previsões com os dados de teste
    previsao_treino = modelo.predict(x_train)
    previsao_teste =  modelo.predict(x_test)
    
    # Calcular as métricas de treino
    mae_train = mean_absolute_error(y_train, previsao_treino)
    rmse_train = np.sqrt(mean_squared_error(y_train, previsao_treino)) 
    r2_train = r2_score(y_train, previsao_treino)
    
    # Calcular as métricas de teste
    mae_test = mean_absolute_error(y_test, previsao_teste)
    rmse_test = np.sqrt(mean_squared_error(y_test, previsao_teste)) 
    r2_test = r2_score(y_test, previsao_teste)
    
    # Concatenar resultados
    resultados = {
        'Métrica': ['R²', 'MAE', 'RMSE'],
        'Treino': [r2_train, mae_train, rmse_train],
        'Teste': [r2_test, mae_test, rmse_test]
    }
    resultado = pd.DataFrame(resultados)
    return round(resultado,3)

## LightGBM

In [ ]:
#Modelo Básico
lgbm_model = lgb.LGBMRegressor(random_state=42)

#Treinando Modelo
lgbm_model.fit(x_train, y_train)

#Avaliando previsões
previsoes = lgbm_model.predict(x_valid)

In [ ]:
#Avaliando resultado
avaliar_modelo(lgbm_model,x_train,y_train,x_valid,y_valid)

## XGBoost

In [ ]:
#Modelo Básico
xgb_model = xgb.XGBRegressor(random_state=42)

#Treinando Modelo
xgb_model.fit(x_train, y_train)

#Avaliando previsões
previsoes = xgb_model.predict(x_valid)

In [ ]:
#Avaliando resultado
avaliar_modelo(xgb_model,x_train,y_train,x_valid,y_valid)

# Interpretando Modelos Estatísticos

In [ ]:
# Habilitar o JavaScript para visualização no notebook
shap.initjs()

In [ ]:
explainer_lgbm = shap.TreeExplainer(lgbm_model)
explainer_xgb = shap.TreeExplainer(xgb_model)

shap_values_lgbm = explainer_lgbm.shap_values(x_valid)
shap_values_xgb = explainer_xgb.shap_values(x_valid)

In [ ]:

shap.summary_plot(shap_values_lgbm, x_valid, plot_type="dot", show=False)



plt.show()